In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

from eli5.sklearn import PermutationImportance



In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars


In [7]:
ls

data/            day2_visualisation.ipynb  LICENSE
day1_meta.ipynb  day3_simple_model.ipynb   README.md


In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [69]:
df.head(20)

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,...,param_moc__cat,param_make__cat,param_kraj-pochodzenia__cat,param_engine-power__cat,param_year__cat,param_serwisowany-w-aso__cat,param_rodzaj-paliwa__cat,param_możliwość-finansowania__cat,param_registered-in-poland__cat,param_transmission__cat,param_miesięczna-rata__cat,param_kod-silnika__cat,param_version__cat,param_opłata-początkowa__cat,param_model-pojazdu__cat,param_pojemność-skokowa__cat,param_kierownica-po-prawej-(anglik)__cat,param_fuel-type__cat,param_vat-free__cat,param_acrylic__cat,param_damaged__cat,param_door-count__cat,param_tuning__cat,param_filtr-cząstek-stałych__cat,param_nr-of-seats__cat,param_kolor__cat,param_engine-capacity__cat,param_pearl__cat,param_stan__cat,param_wersja__cat,param_emisja-co2__cat,param_body-type__cat,param_matowy__cat,param_bezwypadkowy__cat,param_akryl-(niemetalizowany)__cat,param_monthly-payment-value__cat,car_id__cat,param_rok-produkcji__int,param_moc__int,param_pojemność-skokowa__int
0,"[Osobowe, Dacia, Logan, Dacia Logan II (2012-)]","13:23, 5 marca 2018",PLN,Cena Brutto,45100.0,"Krasne 7A - 36-007 Krasne, rzeszowski, Podkarp...",AUTO SPEKTRUM,Dealer,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,...,0,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,0,0,-1,-1,-1,0,-1,-1,0,2018,90,898
2,"[Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-...","15:48, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",28000.0,"Tomaszów Mazowiecki, tomaszowski, Łódzkie",None,Osoba prywatna,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,1,-1,0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,-1,0,-1,-1,1,2011,115,1560
3,"[Osobowe, Porsche, Cayenne, Porsche Cayenne II...","06:30, 4 marca 2018",PLN,"Cena Netto, Do negocjacji, Faktura VAT",229500.0,"Kępińska 24a - 63-640 Bralin, kępiński, Wielko...",P.W. GRIB,Dealer,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,True,True,True,False,True,True,True,True,...,2,-1,1,-1,-1,0,1,0,-1,-1,-1,-1,-1,-1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,2,-1,-1,-1,-1,-1,-1,2,2015,262,3000
4,"[Osobowe, Peugeot, 308, Peugeot 308 T7 (2008-2...","08:46, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",19500.0,"Limanowa, limanowski, Małopolskie",None,Osoba prywatna,False,False,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,False,False,...,3,-1,0,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,3,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,1,3,-1,-1,-1,0,-1,-1,3,2009,110,1560
5,"[Osobowe, Volkswagen, Golf, Volkswagen Golf VI...","20:00, 3 marca 2018",PLN,"Cena Brutto, Faktura VAT",164900.0,"Niciarniana 51/53 

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue

  factorize_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorize_values
  else:
    df[feat + SUFFIX_CAT] = factorize_values

In [10]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

print(len(cat_feats))
cat_feats

151


['created_at__cat',
 'seller_address__cat',
 'seller_name__cat',
 'seller_type__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_poduszka-powietrzna-chroniąca-kolana__cat',
 'feature_kurtyny-powietrzne__cat',
 'feature_klimatyzacja-dwustrefowa__cat',
 'feature_światła-led__cat',
 'feature_czujnik-zmierzchu__cat',
 'feature_elektrycznie-ustawiane-lusterka__cat',
 'feature_asr-(kontrola-trakcji)__cat',
 'feature_poduszka-powietrzna-kierowcy__cat',
 'feature_cd__cat',
 'feature_elektryczne-szyby-przednie__cat',
 'feature_poduszka-powietrzna-pasażera__cat',
 'feature_system-start-stop__cat',
 'feature_światła-do-jazdy-dziennej__cat',
 'feature_komputer-pokładowy__cat',
 'feature_elektryczne-szyby-tylne__cat',
 'feature_klimatyzacja-manualna__cat',
 'feature_tapicerka-welurowa__cat',
 'feature_czujnik-deszczu__cat',
 'feature_światła-przeciwmgielne__cat',
 'feature_ogrzewanie-postojowe__cat',
 'feature_radio-niefabryczne__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_

##model: **DecisionTree**

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [30]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

model: **RandomForest**

In [31]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

model: **XGBoost**

In [87]:
xgb_params = {
    'max_depth': 8,
    'n_esimators': 120,
    'learning_rate': 0.05,
    'seed': 2020,
    'min_child_weight': 2
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[14:54:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:55:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10434.250277403306, 123.19974189075579)

In [0]:
feats = ['param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji__cat', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'feature_kamera-cofania__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa__cat', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat']

In [36]:
len(feats)

20

In [0]:
xgb.XGBRegressor(silent=True, min)

In [37]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[13:23:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:23:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:24:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12716.601646830568, 135.5368139027242)

In [39]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji__int'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
#podmiana cechy roku produkcji z 'kategorialnej' na liczbową
feats2 = ['param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji__int', 'param_moc__cat', 'param_skrzynia-biegów__cat', 'feature_kamera-cofania__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa__cat', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat']

In [43]:
run_model(xgb.XGBRegressor(**xgb_params), feats2) # ponownie uruchamiamy model

[13:41:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:41:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11269.307826125441, 104.47654903287268)

In [0]:
df['param_moc'].unique()

In [0]:
df['param_moc__int']  = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
df['param_moc__int'].unique()

In [0]:
#podmiana cechy mocy silnika z 'kategorialnej' na liczbową
feats3 = ['param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji__int', 'param_moc__int', 'param_skrzynia-biegów__cat', 'feature_kamera-cofania__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa__cat', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat']

In [53]:
run_model(xgb.XGBRegressor(**xgb_params), feats3) # ponownie trenujemy model

[14:01:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:01:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:02:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9648.94241216544, 76.95327879187428)

In [60]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa__int'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' cm3')[0].replace(' ', '')))

In [0]:
#podmiana cechy param_pojemność-skokowa z 'kategorialnej' na liczbową
feats4 = ['param_fuel-type__cat', 'seller_type__cat', 'feature_klimatyzacja-dwustrefowa', 'param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji__int', 'param_moc__int', 'param_skrzynia-biegów__cat', 'feature_kamera-cofania__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa__int', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat']

In [85]:
run_model(xgb.XGBRegressor(**xgb_params), feats4) # ponownie trenujemy model

[14:46:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:46:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:46:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8957.537915738927, 56.20404772100292)

In [0]:
#było (-9495.316847806878, 71.09938023023295)